ACTIVITY 2: Extract and Repurpose Data.

Input Data Set : European Soccer Data.

Output files: "players.csv" , "teams.csv"

players.csv : "id","player_api_id","player_name","player_fifa_api_id","overall_rating","gk_rating","appearances","leagues"

teams.csv : "team_api_id","team_long_name","buildUp","chanceCreation","defense","goals"

In [1]:
install.packages("dplyr")
install.packages("methods")
install.packages("sqldf")

In [4]:
library("dplyr")
library("methods")
library("sqldf")


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

Loading required package: gsubfn
Loading required package: proto
Loading required package: RSQLite


In [5]:
my_db <- src_sqlite("database.sqlite", create = TRUE)

In [6]:
show(my_db)

src:  sqlite 3.11.1 [database.sqlite]
tbls: Country, League, Match, Player, Player_Attributes, sqlite_sequence, Team,
  Team_Attributes


In [7]:
player = tbl(my_db,"Player")
player_attr = tbl(my_db,"Player_Attributes")
country = tbl(my_db,"Country")
league = tbl(my_db,"League")
match = tbl(my_db,"Match")
team = tbl(my_db,"Team")
team_attr = tbl(my_db,"Team_Attributes")

Loading the values to a dataframe.

In [8]:
player = data.frame(player)
player_attr = data.frame(player_attr)
country = data.frame(country)
league = data.frame(league)
match = data.frame(match)
team = data.frame(team)
team_attr = data.frame(team_attr)

In [9]:
#Filtering out columns
new_player = player %>% select(id,player_api_id,player_name,player_fifa_api_id)
#new_player

In [10]:
new_player_attr = player_attr %>% select(player_api_id,overall_rating,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes)

In [11]:
rating_table = new_player_attr %>% group_by(player_api_id) %>% summarise(overall_rating = max(overall_rating))

In [12]:
new_player_attr$gk_rating = new_player_attr$gk_diving + new_player_attr$gk_handling + new_player_attr$gk_kicking + new_player_attr$gk_positioning + new_player_attr$gk_reflexes

In [13]:
gk_table = new_player_attr %>% group_by(player_api_id) %>% summarise(gk_rating = max(gk_rating))

Adding columns to the players dataframe by joining two dataframes.

In [14]:
players = left_join(x=new_player,y=rating_table)
players = left_join(x=players, y = gk_table)

Joining, by = "player_api_id"
Joining, by = "player_api_id"


match1 = Contains details of home players

match2 = Contains details of away players

In [15]:
match1 = match %>% select(league_id,match_api_id,home_player_1,home_player_2,home_player_3,home_player_4,home_player_5,home_player_6,home_player_7,home_player_8,home_player_9,home_player_10,home_player_11)
match1 = na.omit(match1)
match2 = match %>% select(league_id,match_api_id,away_player_1,away_player_2,away_player_3,away_player_4,away_player_5,away_player_6,away_player_7,away_player_8,away_player_9,away_player_10,away_player_11)
match2 = na.omit(match2)

Calculating the appearances of each player

In [27]:
for(i in (1:nrow(players)))
    {
    players$appearances[i] = 0
}

In [38]:
#Change the value of home_player_n and away_player_n 
players_size = nrow(players)
for(i in (1:players_size))
    {
    count = 0
    id = players$player_api_id[i]
    home_count = sum(match1$home_player_11 == id)
    away_count = sum(match1$away_player_11 == id)
    count = home_count + away_count
    players$appearances[i] = players$appearances[i] + count
}
#dim(players)
#colnames(players)

In [ ]:
install.packages("plyr")

In [40]:
library("plyr")

------------------------------------------------------------------------------
You have loaded plyr after dplyr - this is likely to cause problems.
If you need functions from both plyr and dplyr, please load plyr first, then dplyr:
library(plyr); library(dplyr)
------------------------------------------------------------------------------

Attaching package: ‘plyr’

The following objects are masked from ‘package:dplyr’:

    arrange, count, desc, failwith, id, mutate, rename, summarise,
    summarize



Counting the number of leagues for each home player.

In [41]:
home1 = count(match1, vars = c("league_id","home_player_1"))
home2 = count(match1, vars = c("league_id","home_player_2"))
home3 = count(match1, vars = c("league_id","home_player_3"))
home4 = count(match1, vars = c("league_id","home_player_4"))
home5 = count(match1, vars = c("league_id","home_player_5"))
home6 = count(match1, vars = c("league_id","home_player_6"))
home7 = count(match1, vars = c("league_id","home_player_7"))
home8 = count(match1, vars = c("league_id","home_player_8"))
home9 = count(match1, vars = c("league_id","home_player_9"))
home10 = count(match1, vars = c("league_id","home_player_10"))
home11 = count(match1, vars = c("league_id","home_player_11"))

Counting the number of leagues for each away player.

In [42]:
away1 = count(match2, vars = c("league_id","away_player_1"))
away2 = count(match2, vars = c("league_id","away_player_2"))
away3 = count(match2, vars = c("league_id","away_player_3"))
away4 = count(match2, vars = c("league_id","away_player_4"))
away5 = count(match2, vars = c("league_id","away_player_5"))
away6 = count(match2, vars = c("league_id","away_player_6"))
away7 = count(match2, vars = c("league_id","away_player_7"))
away8 = count(match2, vars = c("league_id","away_player_8"))
away9 = count(match2, vars = c("league_id","away_player_9"))
away10 = count(match2, vars = c("league_id","away_player_10"))
away11 = count(match2, vars = c("league_id","away_player_11"))

Initializing the values of league_n to zero.

In [43]:
for(i in (1:nrow(players)))
    {
    players$league1[i] = 0
    players$league2[i] = 0
    players$league3[i] = 0
    players$league4[i] = 0
    players$league5[i] = 0
    players$league6[i] = 0
    players$league7[i] = 0
    players$league8[i] = 0
    players$league9[i] = 0
    players$league10[i] = 0
    players$league11[i] = 0
}

Calculating the number of each league for each player.

Change homen to awayn or awayn to homen 11 times.

Change home_player_n/away_player_n

In [65]:
#change here
size = nrow(home11)
for(j in (1:size))
    {
    #change here 2 times
    id = home11$home_player_11[j]
    #change here
    l = home11$league_id[j]
    index = which(players$player_api_id == id)
    if(l == "1")
    {
        players$league1[index] = players$league1[index] + 1
    }
    if(l == "1729")
        {
        players$league2[index] = players$league2[index] +1
    }
    if(l == "4769")
        {
        players$league3[index] = players$league3[index] +1
    }
    if(l == "7809")
        {
        players$league4[index] = players$league4[index] +1
    }
    if(l == "10257")
        {
        players$league5[index] = players$league5[index] +1
    }
    if(l == "13274")
        {
        players$league6[index] = players$league6[index] +1
    }
    if(l == "15722")
        {
        players$league7[index] = players$league7[index] +1
    }
    if(l == "17642")
        {
        players$league8[index] = players$league8[index] +1
    }
    if(l == "19694")
        {
        players$league9[index] = players$league9[index] +1
    }
    if(l == "21518")
        {
        players$league10[index] = players$league10[index] +1
    }
    if(l == "24558")
        {
        players$league11[index] = players$league11[index] +1
    }
    
}

Calculating the number of leagues for each player by summing all league values.

In [66]:
for(i in (1:nrow(players)))
    {
    players$leagues[i] = players$league1[i] + players$league2[i] + players$league3[i] + players$league4[i] +players$league5[i] + players$league6[i] + players$league7[i] + players$league8[i] + players$league9[i] + players$league10[i] + players$league11[i]
}

Writing PLAYERS INFO to a csv file ("players.csv")

In [67]:
write.csv(players[,c("id","player_api_id","player_name","player_fifa_api_id","overall_rating","gk_rating","appearances","leagues")], file="players.csv",row.names=FALSE)


Refining TEAMS INFO for teams.csv

In [68]:
teams = team %>% select(team_api_id,team_long_name)
for(i in (1:nrow(teams)))
    {
    teams$buildUp = 0
    teams$chanceCreation = 0
    teams$defense = 0
}
for(i in (1:nrow(team_attr)))
    {
    id = team_attr$team_api_id[i]
    index = which(teams$team_api_id == id)
    teams$buildUp[index] = team_attr$buildUpPlaySpeed[i] + team_attr$buildUpPlayDribbling[i] + team_attr$buildUpPlayPassing[i]
    teams$chanceCreation[index] = team_attr$chanceCreationPassing[i] + team_attr$chanceCreationCrossing[i] + team_attr$chanceCreationShooting[i]
    teams$defense[index] = team_attr$defencePressure[i] + team_attr$defenceAggression[i] + team_attr$defenceTeamWidth[i]
}

In [69]:
teams[is.na(teams)] = 0

In [70]:
goal_table = match %>% select(match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal)

In [71]:
#Initializing the value of goals for each team to zero.
for(i in (1:nrow(teams)))
    {
    teams$goals[i] = 0
}

In [72]:
#Determing the home__team_goal and away_team_goal
for(i in (1:nrow(goal_table)))
    {
    home = goal_table$home_team_api_id[i]
    away = goal_table$away_team_api_id[i]
    hgoal = goal_table$home_team_goal[i]
    agoal = goal_table$away_team_goal[i]
    home_index = which(teams$team_api_id == home)
    away_index = which(teams$team_api_id == away)
    teams$goals[home_index] = teams$goals[home_index] + hgoal
    teams$goals[away_index] = teams$goals[away_index] + agoal
}

Writing data to teams.csv file

In [73]:
write.csv(teams[,c("team_api_id","team_long_name","buildUp","chanceCreation","defense","goals")], file="teams.csv",row.names=FALSE)

OUTPUT:

Reading the data from players.csv file

In [74]:
data1 = read.csv(file="players.csv", header=TRUE, sep=",")

Reading the data from teams.csv file

In [75]:
data2 = read.csv(file="teams.csv", header=TRUE, sep=",")

In [76]:
#From players.csv
ans1 = head(data1[order(data1$overall_rating, decreasing= T),], n = 10)
ans2 = head(data1[order(data1$gk_rating, decreasing= T),], n = 10)
ans3 = head(data1[order(data1$appearances, decreasing= T),], n = 10)
ans4 = head(data1[order(data1$leagues, decreasing= T),], n = 10)

In [77]:
#From teams.csv
ans5 = head(data2[order(data2$buildUp, decreasing= T),], n = 10)
ans6 = head(data2[order(data2$chanceCreation, decreasing= T),], n = 10)
ans7 = head(data2[order(data2$defense, decreasing= T),], n = 10)
ans8 = head(data2[order(data2$goals, decreasing= T),], n = 10)

In [78]:
print("Top 10 players by overall rating:")
for(i in (1:10))
    {
    if(i==1)
        {
        name = ans1$player_name[i]
        name = factor(name)
        str1 = name
    }
    else
        {
        name = ans1$player_name[i]
        name = factor(name)
        str1 = paste(str1,name,sep =", ")
    }
}
str1

[1] "Top 10 players by overall rating:"


[1] "Lionel Messi, Cristiano Ronaldo, Gianluigi Buffon, Wayne Rooney, Gregory Coupet, Xavi Hernandez, Alessandro Nesta, Andres Iniesta, Fabio Cannavaro, Iker Casillas"

In [79]:
print("Top 10 goalkeepers by gk rating:")
for(i in (1:10))
    {
    if(i==1)
        {
        name = ans2$player_name[i]
        name = factor(name)
        str2 = name
    }
    else
        {
        name = ans2$player_name[i]
        name = factor(name)
        str2 = paste(str2,name,sep =", ")
    }
}
str2

[1] "Top 10 goalkeepers by gk rating:"


[1] "Gianluigi Buffon, Gregory Coupet, Petr Cech, Iker Casillas, Manuel Neuer, Julio Cesar, Sebastian Frey, Edwin van der Sar, David De Gea, Pepe Reina"

In [86]:
print("Top 10 players by number of appearances:")

for(i in (1:10))
    {
    if(i==1)
        {
        name = ans3$player_name[i]
        name = factor(name)
        str3 = name
    }
    else
        {
        name = ans3$player_name[i]
        name = factor(name)
        str3 = paste(str3,name,sep =", ")
    }
}
str3

[1] "Top 10 players by number of appearances:"


,id,player_api_id,player_name,player_fifa_api_id,overall_rating,gk_rating,appearances,leagues
9994,10009,31293,Steve Mandanda,163705,84,421,147,2
4269,4272,26295,Hugo Lloris,167948,86,425,141,4
10286,10301,31465,Tim Howard,16254,84,418,140,2
9414,9427,42422,Samir Handanovic,162835,84,412,138,2
9955,9970,41097,Stephane Ruffier,167628,NA,NA,138,4
5030,5035,31432,Joe Hart,150724,85,414,137,2


[1] "Steve Mandanda, Hugo Lloris, Tim Howard, Samir Handanovic, Stephane Ruffier, Joe Hart, Gorka Iraizoz Moreno, Cristiano Ronaldo, Borja Valero, Cedric Carrasso"

In [81]:
print("Top 10 players by number of leagues they played in:")
for(i in (1:10))
    {
    if(i==1)
        {
        name = ans4$player_name[i]
        name = factor(name)
        str4 = name
    }
    else
        {
        name = ans4$player_name[i]
        name = factor(name)
        str4 = paste(str4,name,sep =", ")
    }
}
str4

[1] "Top 10 players by number of leagues they played in:"


[1] "Gelson Fernandes, Kevin-Prince Boateng, Angel Di Maria, Timmy Simons, Alfred N'Diaye, Borja Valero, Guillaume Gillet, Xherdan Shaqiri, Cheikhou Kouyate, Kevin de Bruyne"

In [82]:
print("Top 10 teams by build up play:")
for(i in (1:10))
    {
    if(i==1)
        {
        name = ans5$team_long_name[i]
        name = factor(name)
        str5 = name
    }
    else
        {
        name = ans5$team_long_name[i]
        name = factor(name)
        str5 = paste(str5,name,sep =", ")
    }
}
str5

[1] "Top 10 teams by build up play:"


[1] "Carpi, Widzew Łódź, Widzew Łódź, FC Nantes, Podbeskidzie Bielsko-Biała, Lazio, Chievo Verona, Blackburn Rovers, Udinese, Genoa"

In [83]:
print("Top 10 teams by chance creation:")
for(i in (1:10))
    {
    if(i==1)
        {
        name = ans6$team_long_name[i]
        name = factor(name)
        str6 = name
    }
    else
        {
        name = ans6$team_long_name[i]
        name = factor(name)
        str6 = paste(str6,name,sep =", ")
    }
}
str6

[1] "Top 10 teams by chance creation:"


[1] "Lazio, Birmingham City, Falkirk, Hamilton Academical FC, Cagliari, Lech Poznań, Juventus, Bayer 04 Leverkusen, SV Werder Bremen, Roma"

In [84]:
print("Top 10 teams by defense:")
for(i in (1:10))
    {
    if(i==1)
        {
        name = ans7$team_long_name[i]
        name = factor(name)
        str7 = name
    }
    else
        {
        name = ans7$team_long_name[i]
        name = factor(name)
        str7 = paste(str7,name,sep =", ")
    }
}
str7

[1] "Top 10 teams by defense:"


[1] "KSV Roeselare, Odra Wodzisław, Falkirk, FC Barcelona, Lazio, Cagliari, FC Bayern Munich, Burnley, Bayer 04 Leverkusen, SV Werder Bremen"

In [85]:
print("Top 10 teams by number of scored goals:")
for(i in (1:10))
    {
    if(i==1)
        {
        name = ans8$team_long_name[i]
        name = factor(name)
        str8 = name
    }
    else
        {
        name = ans8$team_long_name[i]
        name = factor(name)
        str8 = paste(str8,name,sep =", ")
    }
}
str8

[1] "Top 10 teams by number of scored goals:"


[1] "FC Barcelona, Real Madrid CF, Celtic, FC Bayern Munich, PSV, Ajax, FC Basel, Manchester City, Chelsea, Manchester United"